# Setup
Install dependencies

In [ ]:
!pip install fast-plate-ocr[train,onnx]
!pip install tensorflow[and-cuda]

In [2]:
%env TF_CPP_MIN_LOG_LEVEL=3
%env KERAS_BACKEND=tensorflow

env: TF_CPP_MIN_LOG_LEVEL=3
env: KERAS_BACKEND=tensorflow


In [3]:
# Pre-trained Keras model
!wget -q https://github.com/ankandrew/fast-plate-ocr/releases/download/arg-plates/cct_xs_v1_global.keras

# Plate config (defines image size, character set, etc.)
!wget -q https://github.com/ankandrew/fast-plate-ocr/releases/download/arg-plates/cct_xs_v1_global_plate_config.yaml

# Model architecture config
!wget -q https://github.com/ankandrew/fast-plate-ocr/releases/download/arg-plates/cct_xs_v1_global_model_config.yaml

# Inspecting Dataset
Validate the dataset

In [ ]:
# Change the path to actual path for the dataset
!cp -r "../images/finetune_dataset" ./dataset

In [11]:
# Validate train
!fast-plate-ocr validate-dataset \
  --annotations-file ./dataset/train/annotations.csv \
  --plate-config-file cct_xs_v1_global_plate_config.yaml

E0000 00:00:1765577404.844784   28230 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765577404.938943   28230 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765577405.622290   28230 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765577405.622381   28230 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765577405.622387   28230 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765577405.622395   28230 computation_placer.cc:177] computation placer already registered. Please check linka

In [12]:
# Validate val
!fast-plate-ocr validate-dataset \
  --annotations-file ./dataset/val/annotations.csv \
  --plate-config-file cct_xs_v1_global_plate_config.yaml

E0000 00:00:1765577458.363785   28470 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765577458.380758   28470 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765577458.423288   28470 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765577458.423348   28470 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765577458.423353   28470 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765577458.423357   28470 computation_placer.cc:177] computation placer already registered. Please check linka

In [13]:
# View stats
!fast-plate-ocr dataset-stats \
  --annotations ./dataset/train/annotations.csv \
  --plate-config-file cct_xs_v1_global_plate_config.yaml

E0000 00:00:1765577495.933542   28629 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765577495.947386   28629 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765577495.974910   28629 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765577495.974975   28629 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765577495.974985   28629 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765577495.975010   28629 computation_placer.cc:177] computation placer already registered. Please check linka

# Prepare for training

Visualize and prepare augmentation pipeline.



In [14]:
# We invoke it this way so we can visualize it properly in this notebook
%matplotlib inline
%run -m fast_plate_ocr.cli.visualize_augmentation -- \
      --img-dir ./dataset/train \
      --columns 2 \
      --rows 4 \
      --show-original \
      --num-images 50 \
      --plate-config-file cct_xs_v1_global_plate_config.yaml

In [15]:
# Augmentation Pipeline
import albumentations as A
import cv2

A.save(
    A.Compose(
        [
            A.Affine(
                translate_percent=(-0.02, 0.02),
                scale=(0.75, 1.10),
                rotate=(-15, 15),
                border_mode=cv2.BORDER_CONSTANT,
                fill=(0, 0, 0),
                shear=(0.0, 0.0),
                p=0.75,
            ),
            A.RandomBrightnessContrast(brightness_limit=0.10, contrast_limit=0.10, p=0.5),
            A.OneOf(
                [
                    A.HueSaturationValue(
                        hue_shift_limit=5, sat_shift_limit=10, val_shift_limit=10, p=0.7
                    ),
                    A.RGBShift(r_shift_limit=10, g_shift_limit=10, b_shift_limit=10, p=0.3),
                ],
                p=0.3,
            ),
            A.RandomGamma(gamma_limit=(95, 105), p=0.20),
            A.ToGray(p=0.05),
            A.OneOf(
                [
                    A.GaussianBlur(sigma_limit=(0.2, 0.5), p=0.5),
                    A.MotionBlur(blur_limit=(3, 3), p=0.5),
                ],
                p=0.2,
            ),
            A.OneOf(
                [
                    A.GaussNoise(std_range=(0.01, 0.03), p=0.2),
                    A.MultiplicativeNoise(multiplier=(0.98, 1.02), p=0.1),
                    A.ISONoise(intensity=(0.005, 0.02), p=0.1),
                    A.ImageCompression(quality_range=(55, 90), p=0.1),
                ],
                p=0.3,
            ),
            A.OneOf(
                [
                    A.CoarseDropout(
                        num_holes_range=(1, 14),
                        hole_height_range=(1, 5),
                        hole_width_range=(1, 5),
                        p=0.2,
                    ),
                    A.PixelDropout(dropout_prob=0.02, p=0.3),
                    A.GridDropout(ratio=0.3, fill="random", p=0.3),
                ],
                p=0.5,
            ),
        ]
    ),
    filepath_or_buffer="custom_augmentation.yaml",
    data_format="yaml",
)

# Training
Let's train the model!

In [19]:
!ls dataset/val/

annotations.csv  images


In [23]:
!fast-plate-ocr valid \
  --model ./cct_xs_v1_global.keras \
  --plate-config-file ./cct_xs_v1_global_plate_config.yaml \
  --annotations ./dataset/train/annotations.csv

E0000 00:00:1765578503.692601   32849 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765578503.700186   32849 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765578503.718563   32849 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765578503.718606   32849 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765578503.718610   32849 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765578503.718614   32849 computation_placer.cc:177] computation placer already registered. Please check linka

In [31]:
!fast-plate-ocr valid \
  --model ./cct_xs_v1_global.keras \
  --plate-config-file ./cct_xs_v1_global_plate_config.yaml \
  --annotations ./dataset/val/annotations.csv

E0000 00:00:1765588459.187107   73461 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765588459.200683   73461 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765588459.268359   73461 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765588459.268442   73461 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765588459.268448   73461 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765588459.268453   73461 computation_placer.cc:177] computation placer already registered. Please check linka

In [26]:
!fast-plate-ocr train \
  --model-config-file ./cct_xs_v1_global_model_config.yaml \
  --plate-config-file ./cct_xs_v1_global_plate_config.yaml \
  --annotations ./dataset/train/annotations.csv \
  --val-annotations ./dataset/val/annotations.csv \
  --augmentation-path custom_augmentation.yaml \
  --epochs 30 \
  --batch-size 32 \
  --output-dir trained_models/ \
  --weights-path cct_xs_v1_global.keras \
  --label-smoothing 0.0 \
  --weight-decay 0.0005 \
  --lr 0.0005

E0000 00:00:1765579329.866660   36371 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765579329.874166   36371 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765579329.893466   36371 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765579329.893506   36371 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765579329.893511   36371 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765579329.893515   36371 computation_placer.cc:177] computation placer already registered. Please check linka